In [2]:
import pandas as pd
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
from tqdm.asyncio import tqdm_asyncio
from tqdm impot tqdm

nest_asyncio.apply()

client_id = ""
client_secret = ""
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
}

# 좌표 (경도, 위도)를 주소로 변환하는 비동기 함수
async def reverse_geocoding(lat, lng, session):
    output = "json"
    coords = f"{lng},{lat}"
    orders = 'legalcode,admcode,addr,roadaddr'
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    url = f"{endpoint}?coords={coords}&output={output}&orders={orders}"
    
    for i in range(2):
        try:
            async with session.get(url, headers=headers) as res:
                results = await res.json()
            results = results['results']
            if len(results) == 0:
                return None
            area1 = results[0]['region']['area1']['name']
            area2 = results[0]['region']['area2']['name']
            area3 = results[0]['region']['area3']['name']
            area4 = results[0]['region']['area4']['name']
            addr = area1 + " " + area2 + " " + area3 + " " + area4
            return addr
        except Exception as e:
            return None
    return None

# 엑셀 파일에서 데이터 로드 후 좌표 정보를 이용해 주소 정보 생성
df = pd.read_excel('2023_end.xlsx')
df['lat_lng'] = df.apply(lambda row: (row['lat'], row['lng']), axis=1)
addresses = []

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.ensure_future(reverse_geocoding(*lat_lng, session)) for lat_lng in df['lat_lng']]
        addresses = []
        for address in tqdm_asyncio.as_completed(tasks, total=len(tasks)):
            addr = await address
            addresses.append(addr)
        df['address'] = addresses
        df.to_excel('geodata_result_end.xlsx', index=False)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


100%|██████████| 259480/259480 [05:42<00:00, 757.97it/s] 
